In [1]:
import pandas as pd
import numpy as np
from datetime import date
from statistics import mean, mode
from pandas.api.types import CategoricalDtype

In [2]:
today = date.today()
print(today)

2023-09-03


# Tables

In [3]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [4]:
# tables
filename_careerorglink = "careerorglink.xlsx"
filename_leadercareerlink = "leadercareerlink.xlsx"
filename_orgtree = "orgtree.xlsx"
filename_elected = "positions_elected.xlsx"

In [5]:
col = pd.read_excel(path_tables + filename_careerorglink,dtype="str")
col.shape

(9002, 12)

In [6]:
col.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position',
       'Notes'],
      dtype='object')

In [7]:
lcl = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lcl.shape

(12617, 3)

In [8]:
lcl.columns

Index(['LeaderID', 'CareerString', 'CareerDateString_2022'], dtype='object')

In [9]:
org = pd.read_excel(path_tables + filename_orgtree,dtype="str")
org.shape

(2368, 19)

In [10]:
org.columns

Index(['InstitutionType', 'OrgType', 'PrimaryInstitution', 'OrgName',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'Alias_OrgName',
       'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year', 'Notes',
       'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index'],
      dtype='object')

In [11]:
elected = pd.read_excel(path_tables + filename_elected,dtype="str")
elected.shape

(105, 4)

In [12]:
elected.columns

Index(['PrimaryInstitution', 'OrgName', 'Position', 'IsElected'], dtype='object')

# Queries

In [13]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [14]:
filename_leaderjobtransition = "leaderjobtransition_alljobs.xlsx"
# filename_leaderjobtransition = "leaderjobtransition_ingov.xlsx"

In [15]:
trans = pd.read_excel(path_queries + filename_leaderjobtransition,dtype="str")
trans.shape

(2868, 27)

#### change datatypes

In [16]:
trans = trans.astype({"OrgAdvance":"int","PositionAdvance":"int","CareerStartYear_1":"int"})
trans.dtypes

LeaderID                   object
CareerString_1             object
CareerDateString_2022_1    object
CareerStartYear_1           int32
CareerStartMonth_1         object
CareerStartDate_1          object
CareerSubstring_1          object
InstitutionType_1          object
PrimaryInstitution_1       object
OrgName_1                  object
Position_1                 object
OrgRank_1                  object
PositionRank_1             object
CareerString_2             object
CareerDateString_2022_2    object
CareerStartYear_2          object
CareerStartMonth_2         object
CareerStartDate_2          object
CareerSubstring_2          object
InstitutionType_2          object
PrimaryInstitution_2       object
OrgName_2                  object
Position_2                 object
OrgRank_2                  object
PositionRank_2             object
OrgAdvance                  int32
PositionAdvance             int32
dtype: object

#### filter out UNCERTAIN - I thought I already did this

In [17]:
certain_criteria = (trans["InstitutionType_1"]!="UNCERTAIN") & (trans["InstitutionType_2"]!="UNCERTAIN")
trans = trans[certain_criteria]
trans.shape

(2774, 27)

# Analysis

In [18]:
path_analysis = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/analysis/"

In [19]:
# subpaths
current_subpath = "2023.09.01 analysis/"

In [20]:
filename_suffix = "_"+filename_leaderjobtransition.split("_")[1]

# Functions

In [21]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

In [22]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

# Step. Org & Position Advance, overall

#### new Variables

In [23]:
trans["SameInstitutionType"] = trans.apply(lambda x: x["InstitutionType_1"] == x["InstitutionType_2"],axis=1)

In [24]:
trans.groupby(["SameInstitutionType"])["LeaderID"].count()

SameInstitutionType
False    1212
True     1562
Name: LeaderID, dtype: int64

#### Overall mean OrgAdvance, PositionAdvance

- but these don't mean much, because OrgRank are not equivalent across Institutions

In [25]:
mean(list(trans.OrgAdvance))

0.05803893294881038

In [26]:
mean(list(trans.PositionAdvance))

0.06560922855082912

#### Overall mean OrgAdvance, PositionAdvance, by SameInstitutionType

In [27]:
h1 = trans.groupby("SameInstitutionType",as_index=False).agg({"LeaderID":"count","OrgAdvance":["mean","var"],"PositionAdvance":["mean","var"]})
h1

SameInstitutionType LeaderID OrgAdvance           PositionAdvance          
                         count       mean       var            mean       var
0               False     1212   0.043729  1.754486        0.034653  0.895578
1                True     1562   0.069142  0.863890        0.089629  0.662045

In [28]:
filename_leaderjobtransition

'leaderjobtransition_alljobs.xlsx'

In [29]:
filename_stem = "h1_overall.xlsx"
h1.to_excel(path_analysis + current_subpath + filename_stem)

# H1. OrgAdvance, PositionAdvance by Institution

In [30]:
trans_h1 = trans[(trans["SameInstitutionType"]==True) & ~(trans["InstitutionType_1"]=="UNCERTAIN")]
trans_h1.shape

(1562, 28)

In [31]:
trans_h1.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Position_1', 'OrgRank_1', 'PositionRank_1',
       'CareerString_2', 'CareerDateString_2022_2', 'CareerStartYear_2',
       'CareerStartMonth_2', 'CareerStartDate_2', 'CareerSubstring_2',
       'InstitutionType_2', 'PrimaryInstitution_2', 'OrgName_2', 'Position_2',
       'OrgRank_2', 'PositionRank_2', 'OrgAdvance', 'PositionAdvance',
       'SameInstitutionType'],
      dtype='object')

In [32]:
trans_h1["InstitutionType"]=np.nan

trans_h1.loc[trans_h1["InstitutionType_1"]=="노동당","InstitutionType"]="Party"
trans_h1.loc[trans_h1["InstitutionType_1"]=="인민군","InstitutionType"]="Military"
trans_h1.loc[trans_h1["InstitutionType_1"]=="정권기관","InstitutionType"]="Government"

nongov_criteria = (trans_h1["InstitutionType_1"]!="노동당") & (trans_h1["InstitutionType_1"]!="인민군") & (trans_h1["InstitutionType_1"]!="정권기관")
trans_h1.loc[nongov_criteria,"InstitutionType"]="Non-Governmental Organization"

C:\Users\seoul\AppData\Local\Temp\ipykernel_13700\292332459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_h1["InstitutionType"]=np.nan


In [33]:
institutionalorder = CategoricalDtype(["Party","Military","Government","Non-Governmental Organization"],ordered=True)
trans_h1['InstitutionType'] = trans_h1['InstitutionType'].astype(institutionalorder)
trans_h1 = trans_h1.sort_values("InstitutionType")

C:\Users\seoul\AppData\Local\Temp\ipykernel_13700\3139864340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_h1['InstitutionType'] = trans_h1['InstitutionType'].astype(institutionalorder)


In [34]:
trans_h1_results = trans_h1.groupby(["InstitutionType"],as_index=False).agg({"LeaderID":"count","OrgAdvance":["mean","var"],"PositionAdvance":["mean","var"]})
trans_h1_results

InstitutionType LeaderID OrgAdvance            \
                                    count       mean       var   
0                          Party      263   0.060837  0.988651   
1                       Military      167   0.203593  2.367939   
2                     Government      834   0.076739  0.776817   
3  Non-Governmental Organization      298  -0.020134  0.147741   

  PositionAdvance            
             mean       var  
0        0.007605  0.648797  
1        0.017964  0.668350  
2        0.100719  0.700528  
3        0.171141  0.553104

#### org advancement by institution

In [35]:
h1_institution_org = trans_h1.groupby(["InstitutionType"],as_index=False).agg({"LeaderID":"count","OrgAdvance":["mean","var"]})
h1_institution_org.columns = ["InstitutionType","count","mean","var"]
h1_institution_org

,InstitutionType,count,mean,var
0,Party,263,0.060837,0.988651
1,Military,167,0.203593,2.367939
2,Government,834,0.076739,0.776817
3,Non-Governmental Organization,298,-0.020134,0.147741


In [36]:
h1_institution_org["sd"] = np.sqrt(h1_institution_org["var"] / (h1_institution_org["count"] - 1))
h1_institution_org["z"] = 1.65
h1_institution_org["e"] = h1_institution_org["sd"] / np.sqrt(h1_institution_org["count"] - 1)
h1_institution_org["lower 95% bound"] = h1_institution_org["mean"] - (h1_institution_org["z"]*h1_institution_org["e"])
h1_institution_org["upper 95% bound"] = h1_institution_org["mean"] + (h1_institution_org["z"]*h1_institution_org["e"])
h1_institution_org

,InstitutionType,count,mean,var,sd,z,e,lower 95% bound,upper 95% bound
0,Party,263,0.060837,0.988651,0.061429,1.65,0.003795,0.054575,0.067098
1,Military,167,0.203593,2.367939,0.119435,1.65,0.009270,0.188297,0.218888
2,Government,834,0.076739,0.776817,0.030538,1.65,0.001058,0.074993,0.078484
3,Non-Governmental Organization,298,-0.020134,0.147741,0.022303,1.65,0.001294,-0.022270,-0.017999


In [37]:
filename_stem = "h1_institution_org.xlsx"
h1_institution_org.to_excel(path_analysis + current_subpath + filename_stem)

#### position advancement by institution

In [38]:
h1_institution_pos = trans_h1.groupby(["InstitutionType"],as_index=False).agg({"LeaderID":"count","PositionAdvance":["mean","var"]})
h1_institution_pos.columns = ["InstitutionType","count","mean","var"]
h1_institution_pos

,InstitutionType,count,mean,var
0,Party,263,0.007605,0.648797
1,Military,167,0.017964,0.668350
2,Government,834,0.100719,0.700528
3,Non-Governmental Organization,298,0.171141,0.553104


In [39]:
h1_institution_pos["sd"] = np.sqrt(h1_institution_pos["var"] / (h1_institution_pos["count"] - 1))
h1_institution_pos["z"] = 1.65
h1_institution_pos["e"] = h1_institution_pos["sd"] / np.sqrt(h1_institution_pos["count"] - 1)
h1_institution_pos["lower 95% bound"] = h1_institution_pos["mean"] - (h1_institution_pos["z"]*h1_institution_pos["e"])
h1_institution_pos["upper 95% bound"] = h1_institution_pos["mean"] + (h1_institution_pos["z"]*h1_institution_pos["e"])
h1_institution_pos

,InstitutionType,count,mean,var,sd,z,e,lower 95% bound,upper 95% bound
0,Party,263,0.007605,0.648797,0.049763,1.65,0.003074,0.002532,0.012677
1,Military,167,0.017964,0.668350,0.063452,1.65,0.004925,0.009838,0.026090
2,Government,834,0.100719,0.700528,0.028999,1.65,0.001005,0.099062,0.102377
3,Non-Governmental Organization,298,0.171141,0.553104,0.043154,1.65,0.002504,0.167009,0.175273


In [64]:
filename_stem = "h1_institution_pos.xlsx"
h1_institution_pos.to_excel(path_analysis + current_subpath + filename_stem)

# Step. Descriptive Analysis

#### Frequency of InstitutionType

In [41]:
column_names = ["LeaderID","InstitutionType"]

In [42]:
df1_column_names = ["LeaderID"]+[item+"_1" for item in column_names if item!="LeaderID"]
df2_column_names = ["LeaderID"]+[item+"_2" for item in column_names if item!="LeaderID"]
df1 = trans[df1_column_names]
df2 = trans[df2_column_names]
df1.columns = column_names
df2.columns = column_names

In [43]:
# concatentate vars
var = pd.concat([df1,df2])

In [44]:
var.loc[var["InstitutionType"]=="노동당","InstitutionType"]="Party"
var.loc[var["InstitutionType"]=="인민군","InstitutionType"]="Military"
var.loc[var["InstitutionType"]=="정권기관","InstitutionType"]="Government"

nongov_criteria = (var["InstitutionType"]!="Party") & (var["InstitutionType"]!="Military") & (var["InstitutionType"]!="Government")
var.loc[nongov_criteria,"InstitutionType"]="Non-Governmental Organization"
var.groupby(["InstitutionType"],as_index=False).count()

,InstitutionType,LeaderID
0,Government,2427
1,Military,432
2,Non-Governmental Organization,1726
3,Party,963


In [45]:
institutionalorder = CategoricalDtype(["Party","Military","Government","Non-Governmental Organization"],ordered=True)
var['InstitutionType'] = var['InstitutionType'].astype(institutionalorder)
var = var.sort_values("InstitutionType")
var.groupby(["InstitutionType"],as_index=False).count()

,InstitutionType,LeaderID
0,Party,963
1,Military,432
2,Government,2427
3,Non-Governmental Organization,1726


In [46]:
filename_stem = "institutioncounts"
institutioncounts = var.groupby(["InstitutionType"],as_index=False).count()
institutioncounts.to_excel(path_analysis + current_subpath + filename_stem + filename_suffix,index=False)

#### Time Series of Transitions, overall

In [47]:
trans["CareerStartYear_1"]

0       1997
1       1997
2       1998
3       1998
4       1998
        ... 
2863    2018
2864    2017
2865    2020
2866    2017
2867    2019
Name: CareerStartYear_1, Length: 2774, dtype: int32

In [48]:
yeardist = trans[["LeaderID","CareerStartYear_1"]].groupby("CareerStartYear_1",as_index=False).count().sort_values("CareerStartYear_1")

In [49]:
yeardist

,CareerStartYear_1,LeaderID
0,1937,1
1,1945,5
2,1946,6
3,1947,1
4,1948,8
...,...,...
72,2016,45
73,2017,23
74,2018,18
75,2019,30


In [50]:
x = pd.DataFrame({"year":pd.Series(range(1945,2021))})

In [51]:
ts = x.merge(yeardist,left_on="year",right_on="CareerStartYear_1",how="left",indicator=True)
merge_results(ts)


Merge Results...

	shape     : (76, 4)
	left_only : (0, 4)
	both      : (76, 4)
	right_only: (0, 4)


In [52]:
select_columns = ["year","LeaderID"]
ts = ts[select_columns]

In [53]:
filename_stem = "transition_ts.xlsx"
ts.to_excel(path_analysis + current_subpath + filename_stem,index=False)

#### Time Series of within-institution transitions, by institution

In [54]:
trans_h1["CareerStartYear_1"]

2867    2019
589     2010
585     1993
2533    2009
2534    2010
        ... 
1242    2006
700     1965
701     1965
2373    2002
1630    1983
Name: CareerStartYear_1, Length: 1562, dtype: int32

In [55]:
yeardist_h1 = trans_h1[["LeaderID","CareerStartYear_1","InstitutionType"]].groupby(["CareerStartYear_1","InstitutionType"],as_index=False).count().sort_values("CareerStartYear_1")

In [56]:
yeardist_h1.columns = ["year","InstitutionType","LeaderID"]
yeardist_h1

,year,InstitutionType,LeaderID
0,1937,Party,0
1,1937,Military,0
2,1937,Government,0
3,1937,Non-Governmental Organization,1
4,1945,Party,2
...,...,...,...
295,2019,Non-Governmental Organization,1
297,2020,Military,0
298,2020,Government,2
296,2020,Party,13


In [57]:
x = pd.DataFrame({"year":pd.Series(range(1945,2021))})
inst = pd.DataFrame({"InstitutionType":pd.Series(["Party","Military","Government","Non-Governmental Organization"])})
frame = x.merge(inst,how="cross")

In [58]:
ts_key_columns = ["year","InstitutionType"]
ts = frame.merge(yeardist_h1,on=ts_key_columns,how="left",indicator=False)
# merge_results(ts)

In [59]:
ts.LeaderID.unique()

array([ 2.,  0.,  1., nan,  6.,  5.,  3.,  4.,  7.,  8.,  9., 19., 12.,
       16., 10., 15., 11., 32., 25., 22., 21., 14., 13., 18., 33., 51.,
       26., 27., 23., 31., 17.])

In [60]:
# pad empty counts with 0
ts.loc[ts.LeaderID.isna(),"LeaderID"]=0

In [61]:
ts_pivot = ts.pivot(index="year",columns="InstitutionType",values="LeaderID")
# ts_pivot.droplevel('year', axis=1)
# ts_pivot.columns = ["year","Government","Military","Non-Governmental Organization","Party"]
ts_pivot

InstitutionType,Government,Military,Non-Governmental Organization,Party
year,,,,
1945,0.0,0.0,0.0,2.0
1946,0.0,0.0,1.0,1.0
1947,0.0,0.0,0.0,0.0
1948,0.0,6.0,0.0,2.0
1949,1.0,0.0,0.0,0.0
...,...,...,...,...
2016,12.0,7.0,2.0,17.0
2017,8.0,2.0,1.0,5.0
2018,3.0,3.0,0.0,5.0


In [62]:
ts_pivot = ts_pivot[["Party","Military","Government","Non-Governmental Organization"]]
ts_pivot = ts_pivot.reset_index()
ts_pivot

InstitutionType,year,Party,Military,Government,Non-Governmental Organization
0,1945,2.0,0.0,0.0,0.0
1,1946,1.0,0.0,0.0,1.0
2,1947,0.0,0.0,0.0,0.0
3,1948,2.0,6.0,0.0,0.0
4,1949,0.0,0.0,1.0,0.0
...,...,...,...,...,...
71,2016,17.0,7.0,12.0,2.0
72,2017,5.0,2.0,8.0,1.0
73,2018,5.0,3.0,3.0,0.0
74,2019,16.0,0.0,4.0,1.0


In [63]:
filename_stem = "transitions_ts_byinstitution.xlsx"
ts_pivot.to_excel(path_analysis + current_subpath + filename_stem,index=False)